## Voting Configuration

In [1]:
# Import packages

In [2]:
import numpy as np 
import pandas as pd 
import warnings
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
warnings.filterwarnings("ignore")

In [3]:
 # Algorithms
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LogisticRegression

In [4]:
# Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#Cross Validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
# Searching
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [5]:
# Visualization
import plotly as plot
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import accuracy_score,mean_squared_error
import plotly.offline as pyo
from plotly.offline import init_notebook_mode,plot,iplot

In [6]:
# Read Data set 
data = pd.read_csv("New_Dataset_Gender_Height_Weight_Age_Calories.csv")

In [7]:
x = data.iloc[:,:4]
y = data.iloc[:,4]

In [8]:
# Feature scalling
scaler=MinMaxScaler()
x=scaler.fit_transform(x)

In [ ]:
# AdaBoost  Svr
svr=SVR(C=19,kernel='rbf')
svr_adb=AdaBoostRegressor(svr,loss='linear',learning_rate=0.1, n_estimators=100)
svr_s = cross_val_score(svr_adb,x,y, cv=5);
svr_results=np.average(svr_s) ;
print(svr_s)
print(svr_results)

In [ ]:
# AdaBoost Rf
rf= RandomForestRegressor(max_depth=7, max_features='log2', n_estimators=1000)                   
rf_adb =AdaBoostRegressor(rf,learning_rate=0.1, n_estimators=9)                
rf_s = cross_val_score(rf_adb,x,y, cv=5);
rf_results=np.average(rf_s) ;
print(rf_s)
print(rf_results)

In [ ]:
# AdaBoost XGB
xgb= XGBRegressor(alpha=30, booster='gbtree', gamma =0.01, n_estimators=15)  
xgb_adb = AdaBoostRegressor(n_estimators=1, learning_rate=0.30000000000000004)
xgb_s = cross_val_score(xgb ,x,y, cv=5);
xgb_results=np.average(xgb_s) ;
print(xgb_s)
print(xgb_results)

In [12]:
# AdaBoost GB
gb= GradientBoostingRegressor(loss='lad', max_features='log2')
gb_adb = AdaBoostRegressor(n_estimators=7, learning_rate=0.1)
gb_s = cross_val_score(gb ,x,y, cv=5);
gb_results=np.average(gb_s) ;
print(gb_s)
print(gb_results)

[0.47046006 0.74082751 0.74900156 0.76115946 0.89632203]
0.723554121242737


In [13]:
# AdaBoost Linear
lin=LinearRegression(normalize=True,)
lin_adb=AdaBoostRegressor(lin,loss='linear', n_estimators=5) 
lin_s = cross_val_score(lin_adb ,x,y, cv=5);
lin_results=np.average(lin_s) ;
print(lin_s)
print(lin_results)

[0.28012705 0.68302851 0.65666906 0.58944814 0.86656884]
0.6151683200308968


In [14]:
vot = VotingRegressor([('svr_adb',svr_adb),('rf_adb',rf_adb),('xgb',xgb),('gb',gb),('lin_adb',lin_adb)]) #,('adb_xgb',adb_xgb)
cvot= cross_val_score(vot,x,y, cv=5);
vot_results=np.average(cvot);
print(cvot)
print(vot_results)

[0.53921593 0.73780347 0.76018457 0.76211593 0.90462183]
0.7407883469891964


In [ ]:
def printCombination(arr, n, r): 
#creat 1D array
    data = [0]*r; 
 
    combinationUtil(arr, data,0,n - 1, 0, r);                  

def combinationUtil(arr, data, start,end, index, r):  
    sub_comb_list=[];                 
    if (index == r): 
        for j in range(r): 
            #print(data[j], end = " ");
            sub_comb_list.append(data[j]);
        #print(); 
        combinationList.append(sub_comb_list);
        return ; 
     
    i = start;  
    while(i <= end and end - i + 1 >= r - index): 
        data[index] = arr[i]; 
        combinationUtil(arr, data, i + 1,end, index + 1, r);  
        i += 1; 
  
# Driver Code
r=[2,3,4]
arr =  [('svr_adb',svr_adb),('rf_adb',rf_adb),('xgb',xgb),('gb',gb),('lin_adb',lin_adb)]
       
       

n = len(arr); 

combinationList=[];

for i in r:
    printCombination(arr, n, i); 

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from tqdm.notebook import tqdm
from time import sleep

In [ ]:
maxAcc=0;
maxAcc_model=0;

for oneCombination in combinationList: 
    evc = VotingRegressor( estimators= oneCombination);
    model=evc;    
    scores1 = cross_val_score(model,x,y, cv=5);
    results=np.average(scores1) ;
    if results>maxAcc:
        maxAcc=results;
        maxAcc_model=model;
    for bc in oneCombination:
        print(bc[0],end="   ")
    print("    : ",results);

In [ ]:
# svr_adb   xgb   gb   lin_adb       :  0.750408025125199
# svr_adb   rf_adb       :  0.750569943140806
# svr_adb   xgb       :  0.7480920892811702
# svr_adb   gb       :  0.7617938351941704
# svr_adb   rf_adb   gb       :  0.7524403484782376

In [16]:
vot1 = VotingRegressor([('svr_adb',svr_adb),('xgb',xgb),('gb',gb),('lin_adb',lin_adb)]) #,('adb_xgb',adb_xgb) ('rf_adb',rf_adb),('xgb',xgb),('gb',gb),
cvot1= cross_val_score(vot1,x,y, cv=5);
vot1_results=np.average(cvot1);
print(cvot1)
print(vot1_results)

[0.52109455 0.75103709 0.76485921 0.76909773 0.91461471]
0.744140657898988


In [17]:
vot2 = VotingRegressor([('svr_adb',svr_adb), ('rf_adb',rf_adb)]) #,('adb_xgb',adb_xgb) ('rf_adb',rf_adb),('xgb',xgb),('gb',gb),
cvot2= cross_val_score(vot2,x,y, cv=5);
vot2_results=np.average(cvot2);
print(cvot2)
print(vot2_results)

[0.53002608 0.74852895 0.78626444 0.78639963 0.90506529]
0.7512568770977434


In [ ]:
vot3 = VotingRegressor([('svr_adb',svr_adb),('xgb',xgb)]) 
cvot3= cross_val_score(vot3,x,y, cv=5);
vot3_results=np.average(cvot3);
print(cvot3)
print(vot3_results)

In [ ]:
vot4 = VotingRegressor([('svr_adb',svr_adb),('gb',gb)]) 
cvot4= cross_val_score(vot4,x,y, cv=5);
vot4_results=np.average(cvot4);
print(cvot4)
print(vot4_results)

In [ ]:
vot5 = VotingRegressor([('svr_adb',svr_adb),('rf_adb',rf_adb),('gb',gb)]) 
cvot5= cross_val_score(vot5,x,y, cv=5);
vot5_results=np.average(cvot5);
print(cvot5)
print(vot5_results)